# Global Configurations

In [ ]:
import json
import os.path
from os import path, mkdir

from google.colab import drive
drive.mount('/content/drive')

# Global config vars
# Name of Google Drive folder to clone the files into
folder = '/content/drive/MyDrive/finance_git'
folder_contents = ''
static = folder + folder_contents + '/static'
template = folder + folder_contents + '/template'
year = '2022'
currency = '$'
wks_name = 'my-wks'
index_col = 'Month'
year_total = 'Year Total'
ngrok_auth_token = 'my ngrok auth token'
repo_url = 'my repo url'

def print_log(msg, should_log):
  if should_log:
    print(msg)

def read_config(log=False):
  global folder, folder_contents, static, template, year, currency, wks_name,\
    index_col, year_total, ngrok_auth_token, repo_url

  # Create folder if it doesn't exist
  if not path.exists(folder):
    print_log("Creating folder...", log)
    os.mkdir(folder)

  # Load configuration file
  config_file = '/config.json'
  config = {}
  with open(folder + config_file) as f:
    config = json.load(f)

    # Relative path to the git repository (in google drive)
    folder_contents = '/' + config['git_repo_name']

    # Path to the static folder
    static = folder + folder_contents + '/static'
    print_log(f'static path: {static}', log)

    # Path to the template folder
    template = folder + folder_contents + '/template'
    print_log(f'template path: {template}', log)

    # Current year
    year = config['current_year']
    print_log(f'current year: {year}', log)

    # Currency
    currency = config['currency']
    print_log(f'currency: {currency}', log)

    # Name of expenses worksheet
    wks_name = config['wks_name']

    # Name of the index column
    index_col = config['index_col']

    # Name of the year total index
    year_total = config['year_total_col_name']

    # load ngrok auth token
    ngrok_auth_token = config['ngrok_auth_token']

    # load github repository url
    repo_url = config['git_repo_url']

    print_log("Loaded config file successfully!\n", True)

read_config(log=True)

# Installations

## Colab installations

In [ ]:
!pip install flask -q
!pip install pyngrok -q
!pip install plotly==5.13.0 -q
!ngrok authtoken $ngrok_auth_token

## Get files from git into Drive

In [ ]:
from shutil import rmtree
import os.path

target = folder + folder_contents

if path.exists(target):
  rmtree(target)

os.chdir(folder)

# Clone the repository
!git clone $repo_url --quiet

# Clean things up
cleanup_folders = ['/.git', '/.idea']
for cleanup_folder in cleanup_folders:
  if path.exists(target + cleanup_folder):
    rmtree(target + cleanup_folder)

print("Cloned repository successfully!")

# Google Spreadsheet Authorization

In [4]:
from google.colab import auth
import gspread
from google.auth import default

#autenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Utility Functions

In [5]:
def get_categories(df):
  categories = {}
  for index in df.index.values.tolist():
    categories[index] = index
  return categories

def parse(df, start_index, num_cols=None, transpose=False):
  global index_col
  df.columns = df.iloc[start_index]
  output = df.iloc[start_index + 1:start_index + 14]
  if num_cols:
    output = output.iloc[:, :num_cols]
  else:
    output = output.iloc[:, :-1]
  output = output.set_index(index_col)
  currency_regexp = f'[\{currency},]'
  output.iloc[0:13] = output.iloc[0:13].replace(currency_regexp, '', regex=True)
  output = output.astype(float)
  if transpose:
    output = output.transpose()
  return output

def parse_expenses(df):
  return parse(df, 1, transpose=True)

def parse_incomes(df):
  return parse(df, 19, num_cols=6, transpose=True)

def get_plot_as_json(fig):
  return json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)

# Load spreadsheet into Pandas DF

In [ ]:
import pandas as pd
from gspread_dataframe import get_as_dataframe

#defining my worksheet
worksheet = gc.open(f'Finance {year}').worksheet(wks_name)

expenses = ''
exp_cats = ''
incomes = ''
inc_cats = ''

def load_spreadsheet():
  global worksheet, expenses, exp_cats, incomes, inc_cats

  #get_all_values gives a list of rows
  rows = worksheet.get_all_values()

  #Convert to a DataFrame
  df = pd.DataFrame(rows)

  expenses = parse_expenses(df)
  exp_cats = get_categories(expenses)

  incomes = parse_incomes(df)
  inc_cats = get_categories(incomes)

read_config(log=True)
load_spreadsheet()

# Plot Functions

If you wish to add plots, add a python (plotly) function which uses the data extracted in previous cells (you are more then welcome to use the following functions for reference)

In [7]:
import plotly.express as px
import plotly.graph_objects as go

custom_template = {
    "layout": go.Layout(
        legend={
            "font": {
              "family": "Nunito",
              "size": 16,
              "color": "#707070",
            }
        },
        title={
            "font": {
                "size": 28,
                "color": "#1f1f1f"
            },
        },
        plot_bgcolor="#ffffff",
        paper_bgcolor="#ffffff",
        colorway=px.colors.qualitative.G10,
    )
}

def expenses_per_category_pie(month=None):
  data = expenses if month is None else expenses[month]
  fig = px.pie(data,
               values=month if month else year_total,
               names=exp_cats,
               title='Expenses Per Category (Pie)',
               template=custom_template)
  return fig


def expenses_per_category_bar():
  data = expenses.transpose()[:-1].reset_index()
  fig = px.bar(data,
               x='Month',
               y=data.columns.values,
               title='Expenses Per Category (Bar)',
               template=custom_template)
  return fig


def monthly_balance_bar():
  year_total_exp = expenses.sum().reset_index()
  year_total_exp = year_total_exp.rename(columns={0:'Expenses'})
  year_total_exp = year_total_exp.iloc[:12]

  year_total_inc = incomes.sum().reset_index()
  year_total_inc = year_total_inc.rename(columns={0: 'Incomes', 'Month': 'trash'})
  year_total_inc = year_total_inc.iloc[:12]

  data = pd.concat([year_total_exp, year_total_inc], ignore_index=False, axis=1)
  data = data.drop('trash', axis=1)
  data['Balance'] = data.Incomes - data.Expenses

  fig = px.bar(data,
               x='Month',
               y=['Expenses', 'Incomes'],
               hover_name='Month',
               template=custom_template,
               title="Monthly Balance (Bar)",
               labels={
                     "Month": "Month",
                     "value": f"Total ({currency})",
                     "variable": "Type"
                  },
               hover_data={
                   'Balance': True,
                   'variable': False,
                   'Month': False,
                   'value': True
               },
               color_discrete_map={
                   'Expenses': 'lightcoral',
                   'Incomes': 'lightgreen'
               })
  return fig

def monthly_balance_line():
  year_total_exp = expenses.sum().reset_index()
  year_total_exp = year_total_exp.rename(columns={0:'Expenses'})
  year_total_exp = year_total_exp.iloc[:12]

  year_total_inc = incomes.sum().reset_index()
  year_total_inc = year_total_inc.rename(columns={0: 'Incomes', 'Month': 'trash'})
  year_total_inc = year_total_inc.iloc[:12]

  data = pd.concat([year_total_exp, year_total_inc], ignore_index=False, axis=1)
  data = data.drop('trash', axis=1)
  data['Balance'] = data.Incomes - data.Expenses

  fig = px.line(data,
               x='Month',
               y=['Expenses', 'Incomes'],
                markers=True,
               hover_name='Month',
               template=custom_template,
               title="Monthly Balance (Line)",
               labels={
                     "Month": "Month",
                     "value": f"Total ({currency})",
                     "variable": "Type"
                  },
               hover_data={
                   'Balance': True,
                   'variable': False,
                   'Month': False,
                   'value': True
               },
               color_discrete_map={
                   'Expenses': 'lightcoral',
                   'Incomes': 'lightgreen'
               })
  fig.update_traces(line=dict(width=4), marker=dict(size=10))
  return fig

def expense_category_tracker():
  data = expenses.transpose()[:-1].reset_index()
  fig = px.line(data,
                x='Month',
                y=list(get_categories(expenses).values()),
               markers=True,
               template=custom_template,
               title="Expense Tracker (Line)",
               labels={
                     "Month": "Month",
                     "value": f"Total ({currency})",
                     "variable": "Category"
                  },
               color_discrete_map={
                   'Expenses': 'lightcoral',
                   'Incomes': 'lightgreen'
               })
  fig.update_traces(line=dict(width=4), marker=dict(size=10))
  return fig

def monthly_target_line():
  year_total_exp = expenses.sum().reset_index()
  year_total_exp = year_total_exp.rename(columns={0:'Expenses'})
  year_total_exp = year_total_exp.iloc[:12]

  year_total_target = targets.sum().reset_index()
  year_total_target = year_total_target.rename(columns={0: 'Targets', 'Month': 'trash'})
  year_total_target = year_total_target.iloc[:12]

  data = pd.concat([year_total_exp, year_total_target], ignore_index=False, axis=1)
  data = data.drop('trash', axis=1)
  data['Balance'] = data.Incomes - data.Expenses

  fig = px.line(data,
               x='Month',
               y=['Expenses', 'Targets'],
                markers=True,
               hover_name='Month',
               template=custom_template,
               title="Monthly Targets (Line)",
               labels={
                     "Month": "Month",
                     "value": f"Total ({currency})",
                     "variable": "Type"
                  },
               hover_data={
                   'Balance': True,
                   'variable': False,
                   'Month': False,
                   'value': True
               },
               color_discrete_map={
                   'Expenses': 'lightcoral',
                   'Targets': 'lightgreen'
               })
  fig.update_traces(line=dict(width=4), marker=dict(size=10))
  return fig

# Application

If you wish to add a plot to the application, add an endpoint in the following cell (similar to the existing endpoint). Make sure you use the correct function (which you should've created for your plot) and that you add a matching card in the index.html file

In [ ]:
from flask import Flask, render_template, request
from pyngrok import ngrok
# from flask_ngrok import run_with_ngrok
import plotly

app = Flask(__name__,
            template_folder=template,
            static_folder=static)

port = 5000
public_url = ngrok.connect(port).public_url
app.config["BASE_URL"] = public_url

@app.route('/')
def home():
  load_spreadsheet()
  return render_template('index.html')


@app.route('/expenses_per_category_pie')
def expenses_per_category_pie_page():
  monthArg = request.args.get('month')
  month = monthArg if monthArg is not None and monthArg != 'Year' else None
  selectedMonth = 'Year' if month is None else month
  graphJSON = get_plot_as_json(expenses_per_category_pie(month))
  return render_template('plot.html',
                         graphJSON=graphJSON,
                         plotname='Expenses per Category (Pie)',
                         selectedMonth = selectedMonth,
                         monthList=['Year', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'])

@app.route('/expenses_per_category_bar')
def expenses_per_category_bar_page():
  graphJSON = get_plot_as_json(expenses_per_category_bar())
  return render_template('plot.html',
                         graphJSON=graphJSON,
                         plotname='Expenses per Category (Bar)')

@app.route('/monthly_balance_bar')
def monthly_balance_bar_page():
  graphJSON = get_plot_as_json(monthly_balance_bar())
  return render_template('plot.html',
                         graphJSON=graphJSON,
                         plotname='Monthly Balance (Bar)')

@app.route('/monthly_balance_line')
def monthly_balance_line_page():
  graphJSON = get_plot_as_json(monthly_balance_line())
  return render_template('plot.html',
                         graphJSON=graphJSON,
                         plotname='Monthly Balance (Line)')

@app.route('/expense_category_tracker')
def expense_category_tracker_page():
  graphJSON = get_plot_as_json(expense_category_tracker())
  return render_template('plot.html',
                         graphJSON=graphJSON,
                         plotname='Expense Tracker (Line)')

# Run

In [ ]:
print(f'''
======================================================================

To open the app, go to: \"{public_url}\"

======================================================================


''')

app.run()